# News Tweets Classifier
Reference:
- https://iq.opengenus.org/text-classification-using-k-nearest-neighbors/
- https://www.geeksforgeeks.org/saving-a-machine-learning-model/

Mount drive

In [53]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Import library

In [54]:
import numpy as np
import pandas as pd
import pickle
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Import and preprocess dataset 

In [55]:
FILENAME = "/content/drive/My Drive/News Tweets Classifier/news_tweets_labeled.csv"
df = pd.read_csv(FILENAME)
df = df[["text", "category"]]
# df = df.sort_values(["category", "text"], ascending=True)

In [56]:
sw = stopwords.words('english')

wnl = WordNetLemmatizer()
for i in range(df.shape[0]):
  review = re.sub('[^a-zA-Z]', ' ', df.loc[i, 'text'])
  review = review.lower()
  review = review.split()

  review = [wnl.lemmatize(word) for word in review if not word in sw]
  review = ' '.join(review)
  df.loc[i, 'text'] = review

In [57]:
df = df[:3000]
print(df.groupby('category').count())
train_data = df.sample(frac=0.8, random_state=200)
test_data = df.drop(train_data.index)

print(len(train_data))
print(len(test_data))

               text
category           
business        290
entertainment   250
health          283
lifestyle       326
others          808
politics        596
science         126
sport           216
technology       52
weather          53
2400
600


Build dictionary and transform

In [58]:
# Builds a dictionary of features and transforms document to feature vectors and convert tweets to a 
# matrix of token counts (CountVectorizer)
count_vect = CountVectorizer()
x_train_counts = count_vect.fit_transform(train_data['text'])

# Transform a count matrix to a normalized tf-idf representation (tf-idf transformer)
tfidf_transformer = TfidfTransformer()
x_train_tfidf = tfidf_transformer.fit_transform(x_train_counts)

Train the model and give some new tweets

In [59]:
knn = KNeighborsClassifier(n_neighbors=10, weights='distance')

# Train classifier; train_data['category'] will be having numbers assigned for each category in train data
clf = knn.fit(x_train_tfidf, train_data['category'])

# Input data to predict their classes of the given categories
tweets_new = ["Seven people have been arrested following a search of two vehicles in Hayle Cornwall police have said", 
            "RT MoneyTelegraph Your gas and electricity bills are set to increase by 36pc over the next 10 years",
            "The most common symptom of Covid19 is now a headache say experts as they warned people to get tested even if they think they are not suffering from the illness"]
# building up feature vector of input
x_new_counts = count_vect.transform(tweets_new)
# Call transform instead of fit_transform because it's already been fit
x_new_tfidf = tfidf_transformer.transform(x_new_counts)

In [60]:
# Predicting the category of input text: Will give out number of category
predicted = clf.predict(x_new_tfidf)

for doc, category in zip(docs_new, predicted):
  print('%r => %s' %(doc, category))

'Seven people have been arrested following a search of two vehicles in Hayle Cornwall police have said' => others
'RT MoneyTelegraph Your gas and electricity bills are set to increase by 36pc over the next 10 years' => business
'The most common symptom of Covid19 is now a headache say experts as they warned people to get tested even if they think they are not suffering from the illness' => health


Test the model

In [61]:
# use Pipeline to add vectorizer -> transformer -> classifier all in a one compound classifier
tweet_clf = Pipeline([
  ('vect', CountVectorizer()),
  ('tfidf', TfidfTransformer()),
  ('clf', knn)
])

# Fitting train data to the pipeline
tweet_clf.fit(train_data['text'], train_data['category'])

# Test data
docs_test = test_data['text']

# Predicting test data
predicted = tweet_clf.predict(docs_test)
print('We got an accuracy of', np.mean(predicted == test_data['category']) * 100, '% over the test data.')

We got an accuracy of 75.5 % over the test data.


Save the model

In [62]:
classifier_model = pickle.dumps(tweet_clf)